# Week 6 Homework

***Due (pushed to your GitHub branch) on 10/25 by 11:59 pm***

## STAR two-pass alignment

Build the STAR SA:

In [1]:
%mkdir alignment_lemons/
%mkdir alignment_lemons/star/

!STAR \
--runThreadN 32 \
--runMode genomeGenerate \
--genomeDir ../2_genome_exploration/lemons/star \
--genomeFastaFiles ../2_genome_exploration/lemons/lemon_genome.fa \
--sjdbGTFfile ../2_genome_exploration/lemons/lemon_annotations.gtf \
--sjdbOverhang 289 \
--genomeSAindexNbases 13 \
--sjdbGTFfeatureExon CDS

	STAR --runThreadN 32 --runMode genomeGenerate --genomeDir ../2_genome_exploration/lemons/star --genomeFastaFiles ../2_genome_exploration/lemons/lemon_genome.fa --sjdbGTFfile ../2_genome_exploration/lemons/lemon_annotations.gtf --sjdbOverhang 289 --genomeSAindexNbases 13 --sjdbGTFfeatureExon CDS
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Oct 31 17:57:25 ..... started STAR run
Oct 31 17:57:25 ... starting to generate Genome files
Oct 31 17:57:29 ..... processing annotations GTF
Oct 31 17:57:31 ... starting to sort Suffix Array. This may take a long time...
Oct 31 17:57:33 ... sorting Suffix Array chunks and saving them to disk...
Oct 31 17:58:06 ... loading chunks from disk, packing SA...
Oct 31 17:58:14 ... finished generating suffix array
Oct 31 17:58:14 ... generating Suffix Array index
Oct 31 17:59:09 ... completed Suffix Array index
Oct 31 17:59:10 ..... inserting junctions into the genome indices
Oct 31 18:00:28 

Use STAR and ***all*** of your FASTQs for a first-pass alignment. Be sure to update the `sjdbOverhang` parameter to a value that is appropriate for your dataset. Rename the log file so that you can compare the logs from the first and second alignments:

In [2]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/lemons/star \
--readFilesManifest manifest_lemons.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outFileNamePrefix alignment_lemons/star/
!cp alignment_lemons/star/Log.final.out alignment_lemons/star/first-pass.final.out

	STAR --runThreadN 32 --runMode alignReads --genomeDir ../2_genome_exploration/lemons/star --readFilesManifest manifest_lemons.tsv --readFilesCommand zcat --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outFileNamePrefix alignment_lemons/star/
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Oct 31 18:00:33 ..... started STAR run
Oct 31 18:00:34 ..... loading genome
Oct 31 18:00:36 ..... started mapping
Oct 31 21:05:11 ..... finished mapping
Oct 31 21:05:12 ..... started sorting BAM
Oct 31 23:21:05 ..... finished successfully


Use STAR, all of your FASTQs, and `SJ.out.tab` for the second-pass alignment. You will likely need to make your own `manifest.tsv`:

In [4]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/lemons/star \
--readFilesManifest manifest_lemons.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outSAMattributes NH HI AS nM RG \
--outFileNamePrefix alignment_lemons/star/ \
--sjdbFileChrStartEnd alignment_lemons/star/SJ.out.tab

!cp alignment_lemons/star/Log.final.out alignment_lemons/star/second-pass.final.out

	STAR --runThreadN 32 --runMode alignReads --genomeDir ../2_genome_exploration/lemons/star --readFilesManifest manifest_lemons.tsv --readFilesCommand zcat --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outSAMattributes NH HI AS nM RG --outFileNamePrefix alignment_lemons/star/ --sjdbFileChrStartEnd alignment_lemons/star/SJ.out.tab
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Nov 01 00:20:08 ..... started STAR run
Nov 01 00:20:40 ..... loading genome
Nov 01 00:20:46 ..... inserting junctions into the genome indices
Nov 01 00:23:03 ..... started mapping
Nov 01 03:12:36 ..... finished mapping
Nov 01 03:12:37 ..... started sorting BAM
Nov 01 05:28:25 ..... finished successfully


Update this block to briefly describe the differences between the logs for the first and second alignments.

## HISAT alignment

(***Note***: you may need Dr. Wheeler to build the HISAT index of your genome. Ask him for assistance when you get to this part of the assignment.)

Use a Bash `while` loop to align all of your FASTQs and save the output files using the same names as the input FASTQ:

In [ ]:
#hisat2_extract_splice_sites.py lemon_annotations.gtf > hisat/hisat_ss.tsv
#hisat2_extract_exons.py lemon_annotations.gtf > hisat/hisat_exons.tsv
#hisat2-build lemon_genome.fa hisat/genome -p 16 --ss hisat/hisat_ss.tsv --exon hisat/hisat_exons.tsv

In [6]:
%mkdir alignment_lemons/hisat/

!while read -r line; do \
    fq=$(echo $line | awk '{print $1}'); \
    bn=$(basename "$fq" .fastq.gz); \
    rg=$(echo $line | awk '{print $3}' | sed 's/ID://' ); \
    echo "Aligning $fq with $rg"; \
    hisat2 /data/users/corwinbm5021/BIOL343/2_genome_exploration/lemons/hisat/genome -p 32 -U $fq --rg-id $rg --rg SM:$rg --summary-file alignment_lemons/hisat/$bn.log --new-summary > alignment_lemons/hisat/$bn.sam; \
    done < manifest_lemons.tsv

mkdir: cannot create directory ‘alignment_lemons/hisat/’: File exists
Aligning /data/users/corwinbm5021/BIOL343/5_fastq/trimmed_lemons/SRR26560370.fastq.gz with YCQ_1
/bin/bash: hisat2: command not found
Aligning /data/users/corwinbm5021/BIOL343/5_fastq/trimmed_lemons/SRR26560371.fastq.gz with ACH_3
/bin/bash: hisat2: command not found
Aligning /data/users/corwinbm5021/BIOL343/5_fastq/trimmed_lemons/SRR26560372.fastq.gz with ACH_2
/bin/bash: hisat2: command not found
Aligning /data/users/corwinbm5021/BIOL343/5_fastq/trimmed_lemons/SRR26560373.fastq.gz with ACH_1
/bin/bash: hisat2: command not found
Aligning /data/users/corwinbm5021/BIOL343/5_fastq/trimmed_lemons/SRR26560374.fastq.gz with YE72_3
/bin/bash: hisat2: command not found
Aligning /data/users/corwinbm5021/BIOL343/5_fastq/trimmed_lemons/SRR26560375.fastq.gz with YE72_2
/bin/bash: hisat2: command not found
Aligning /data/users/corwinbm5021/BIOL343/5_fastq/trimmed_lemons/SRR26560376.fastq.gz with YE72_1
/bin/bash: hisat2: command

## Snakemake Pipeline

In contrast to last week (FASTQ files), there aren't as many decisions that need to be made during alignment, so using a notebook to control and report code isn't as necessary. As you saw in in [6_alignment](6_alignment.ipynb), file and directory management can get complicated during alignment. This is a case where Snakemake scripting shines.

1. Copy your `Snakefile` and `config.yaml` from `5_fastq` to `6_alignment`. 
2. In the `Snakefile`, add the following rules:  
    a. `fetch_genome` - using a variation from the code from `2_genome_exploration`, download your reference genome and annotations.  
    b. `star_sa` - use `STAR` to produce the suffix array of the reference genome.  
    c. `star_align_first` - use `STAR` to perform first-pass alignment with all FASTQ files.  
    d. `star_align_second` - use `STAR` to perform second-pass alignment with all FASTQ files.  
    e. `hisat_align` - use `HISAT` to align all FASTQ files.  
    f. `report` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.